In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

## Get devtools

In [3]:
json_path = "https://raw.githubusercontent.com/ethereum-optimism/Retro-Funding/refs/heads/main/results/S7/m1_devtooling_results.json"
df_devtools = pd.read_json(json_path)
df_devtools.head(3)

,project_id,project_name,display_name,round_id,is_eligible,op_reward,star_count,fork_count,num_packages_in_deps_dev,package_connection_count,developer_connection_count,onchain_builder_oso_project_ids,onchain_builder_op_atlas_ids,trusted_developer_usernames,onchain_builder_op_atlas_names
0,FT3wZj26UAebMH4ywu7oQvrie1GQSmu9aogl40bsCWQ=,0x0008577196fa6ec286440b418e2f6305fc10e62ce759...,CreateX,7,True,4408.20,233,34,0,0,21,"[g6Z4CSsse62oS12pTQrmQ5gu1iLt74uJP5rwFgdhX1c=,...",[0x7262ed9c020b3b41ac7ba405aab4ff37575f8b6f975...,"[pegahcarter, derekbar90, avious00, welps, sud...","[Aave, Aragon OSx, Rainbow (Rainbow Router), S..."
1,T5JyiUTt5UnCt95/6Xyjn04da6UU9epzDAizTapPUUU=,0x07222a857f4522026d91faec337e3cb8cf6ec5f7afb8...,Impersonator.xyz,7,True,1418.48,75,23,0,0,5,"[GFtPmpIWk2qUtklRVLF1k8VATmNH+3bHG4CA8iOfuxA=,...",[0x5614d129eba060ce7d224e8e8f9f29aeddfdb572b0d...,"[0xbigboss, chase-manning, pcaversaccio, murde...",[Send]
2,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be...,Velodrome Finance,7,True,1365.90,105,63,0,0,9,"[Jpscljo0tkzVDYKwVNnTSuR+AFUyy4EHg4AWd+Mdnd4=,...",[0x02065e72fe4eebfa1ebca19238e6147c8571c2b7fef...,"[pegahcarter, fringlesinthestreet, 0xmotto, cr...","[Exa App, Rainbow (Rainbow Router)]"


In [4]:
OSO_IDS = list({_id for lst in df_devtools['onchain_builder_oso_project_ids'] for _id in lst})
ATLAS_IDS = list({_id for lst in df_devtools['onchain_builder_op_atlas_ids'] for _id in lst})
METRICS = [
    f"{c.upper()}_gas_fees_daily"
    for c in [
        'automata', 'cyber', 'ham', 
        'base', 'bob', 'frax', 'ink', 'kroma', 'lisk', 'lyra',
        'metal', 'mint', 'mode', 'optimism', 'orderly', 'polynomial',
        'race', 'redstone', 'shape', 'soneium', 'swan', 'swell',
        'unichain', 'worldchain', 'xterio', 'zora'
    ]
]

## Lookup Gas Fees

In [5]:
df_onchain_metrics_oso = client.to_pandas(f"""

select
    p.project_id,
    p.project_name,
    p.display_name,
    SUM(tm.amount) as feb_gas_fees
from oso.timeseries_metrics_by_project_v0 tm
join oso.metrics_v0 m on tm.metric_id = m.metric_id
join oso.projects_v1 p on tm.project_id = p.project_id
where    
    tm.sample_date >= DATE('2025-02-01') AND tm.sample_date < DATE('2025-03-01')    
    and p.project_id IN ({stringify(OSO_IDS)})
    and m.metric_name IN ({stringify(METRICS)})
group by 1,2,3
order by 4 desc

""")

In [6]:
df_onchain_metrics_atlas = client.to_pandas(f"""

select
    p.project_id,
    p.project_name,
    p.display_name,
    SUM(tm.amount) as feb_gas_fees
from oso.timeseries_metrics_by_project_v0 tm
join oso.metrics_v0 m on tm.metric_id = m.metric_id
join oso.projects_v1 p on tm.project_id = p.project_id
where    
    tm.sample_date >= DATE('2025-02-01') AND tm.sample_date < DATE('2025-03-01')    
    and p.project_name IN ({stringify(ATLAS_IDS)})
    and m.metric_name IN ({stringify(METRICS)})
group by 1,2,3
order by 4 desc

""")

In [7]:
gas_by_oso_id = df_onchain_metrics_oso.set_index('project_id')['feb_gas_fees'].to_dict()
gas_by_atlas_id = df_onchain_metrics_atlas.set_index('project_name')['feb_gas_fees'].to_dict()

In [8]:
cols = ['project_name', 'display_name', 'onchain_builder_oso_project_ids', 'onchain_builder_op_atlas_ids']
df = df_devtools[cols].copy()

df['oso_gas_fees'] = df_devtools['onchain_builder_oso_project_ids'].apply(
    lambda lst: sum(gas_by_oso_id.get(_id,0) for _id in lst)
)
df['atlas_gas_fees'] = df_devtools['onchain_builder_op_atlas_ids'].apply(
    lambda lst: sum(gas_by_atlas_id.get(_id,0) for _id in lst)
)
df['fee_ratio'] = df['oso_gas_fees'] / df['atlas_gas_fees']

df.query("atlas_gas_fees > 0").sort_values(by='fee_ratio')

,project_name,display_name,onchain_builder_oso_project_ids,onchain_builder_op_atlas_ids,oso_gas_fees,atlas_gas_fees,fee_ratio
40,0x4e175118c15129e6cc791203de92b23f11fef35ba4bc...,EvmTools,[hE1BjGS8I9OXjnYATSkRnNJlsItBXO+CKXU6UPKJlRQ=],[0x3061b642db56c507e265f03029735b0413a613bda43...,2.011750,2.011750,1.000000e+00
49,0x537097f27d55cb52d39b95c3ed65076349ab68aac480...,xdeployer,"[ckq4jTDZI8VgQseutflTvtvIG5SawYliPbHPn3thsIw=,...",[0x70ba3cb2e433f1cd9ace8aea84c46d167ba5db12293...,677.642495,670.497123,1.010657e+00
94,0x9d93ec97ef2d3bd4c2b8d95abac9ce0cf43e4f3eb1f0...,Frames.js,"[iOfFhoUyVo3E06Q1rH0JA6xoj/6XDj+7JQsnopmyhHQ=,...",[0x5614d129eba060ce7d224e8e8f9f29aeddfdb572b0d...,2.192345,2.025524,1.082360e+00
121,0xc377ed1b705bcc856a628f961f1e7c8ca943e6f3727b...,MerkleTreeJS,"[ZzH9XQNZ7nWOlugpkz8GaQQEmQzx1rKxaPrvhJ+l87Y=,...",[0xbcb5c437d5347be21add3972512679054416805274f...,1264.807909,728.688279,1.735732e+00
78,0x819775803938d78eaa95809971ce94cae6a54b4df585...,Vscode Solidity Extension,"[sw8ehEEk8y0FK2GSDVnFLz7M9erX6RSQdSAgGDmiwEs=,...",[0xa2a580dae44336df1cbcb7a5303880f393068fb9611...,15.616746,8.281891,1.885650e+00
...,...,...,...,...,...,...,...
10,0x154a42e5ca88d7c2732fda74d6eb611057fc88dbe6f0...,🏗️ Scaffold-ETH 2,"[v2QgByGF4okeRHMJmWN3tqZMxciee/qI2eeSPKK19gk=,...",[0xd3d00c946f62ac8cf49a85e3ab7d295d3b56f68d0b9...,402.482156,0.434317,9.267021e+02
42,0x4f8902ac6eba8e2243237247302ab778a1e1f496cf62...,blocksmith.js,"[uAWTL5naoNXYNpp7weim8ypopqmN4+H9fO7/1wOjpeA=,...",[0x800707befd6c94e7cea794200a6fee1155e54ca94bc...,12.884087,0.010967,1.174832e+03
53,0x5cfdb8c453a7eede1946071e9d7417d01cc2116f5756...,Optimism × Py-libp2p,"[TsV9kZbHU7k/zigTlg8d7fFUcJ8Kn8YkaluyCauxUtM=,...",[0x149bc5c396fd3dccac39c92164bb6b8bccc4d215321...,13.396920,0.009271,1.445007e+03
123,0xc71faa1bcb4ceb785816c3f22823377e9e5e7c48649b...,go-libp2p,"[hHje/yl10ng5XbAXgyeDEq3hxmOkjscMbS5XNqeAg7Q=,...",[0x149bc5c396fd3dccac39c92164bb6b8bccc4d215321...,160.417651,0.009271,1.730283e+04


In [11]:
df.iloc[121].to_dict()

{'project_name': '0xc377ed1b705bcc856a628f961f1e7c8ca943e6f3727b7c179c657e227e8e852c',
 'display_name': 'MerkleTreeJS',
 'onchain_builder_oso_project_ids': ['ZzH9XQNZ7nWOlugpkz8GaQQEmQzx1rKxaPrvhJ+l87Y=',
  'c/dKIiJnIKIxWPUPaAC30nS/TmqojvpagHXzV2L6td8=',
  'VCeNot3diK3Kt0Da9SaRDvcNI1sGdrzyjTW62JUeI0U=',
  'XREPTQE3axWeuZSDPwZorBfXWwCvrvu1FzVjQsnxEHU=',
  '87Uygt4CNKHiBhvGlVmFtfsKiPrxJiEXWcRnubfQUpg=',
  'RtKY/jR/ZQZcFZ3+6BIol7NCIenNffgO59/55hceklc=',
  'lY3hN7xGe8jTGb9c9z2Sa6r2PmBV+FG9WJmC8Xwe46s=',
  'R/brPbiaD8+ZOgc2k4NVBYT3oWYh50xHrkVT0zKFP94=',
  'E9H3Htqh4/dCGgu9GUI+ksk7cayGewmtONFiBQWlveo=',
  'kCF+D9wyJM2PvCFVlX9aKksPQmQEIKOsEnQUW3Bipog=',
  '8/YZC7L8xkH8nVh1zywNf0hccJAYI09brdiIW1eizWg=',
  '1PqWi/6kQu3j9O311xKAqoA9vYRJjIlLefadHnswYrE=',
  '2l+DcexWOerkhRsKXiaWZam/soE6IQ5tVe7t4K202lo=',
  'xyJHxLgvqigps6mdxRScLr8Gs8Xqs1UE/5JC7l6r/J0=',
  'UP1CZduWIgDJysUtiSKzySeyLCf8ZoZD5PdWYPKiOIw=',
  'SbFbagHrVLpnhXLPsZg6EYdenq1+y5k/aOkGXo32/Ww=',
  'hMNrmX7nfpMimNe0zQmqdkZVy4PrhU3MIliN9gVR9y